In [4]:
import mysql.connector
from apify_client import ApifyClient
import pandas as pd
from sqlalchemy import create_engine
client = ApifyClient("##############################")

In [110]:
def BookingScrapper(city,max):
    run_input = {
        "search": city,
        "maxPages": max,
        "sortBy": "distance_from_search",
        "currency": "USD",
        "language": "en-us",
        "proxyConfig": {"useApifyProxy": True},
        "extendOutputFunction": "($) => { return {} }",
    }
    run = client.actor("dtrungtin/booking-scraper").call(run_input=run_input)
    return client.dataset(run["defaultDatasetId"]).iterate_items()
def TripadvisorScrapper(city,max):
    run_input = {
    "locationFullName": city,
    "maxItems": max,   
    "language": "en",
    "currency": "USD",
    "proxyConfiguration": { "useApifyProxy": True },
    }
    run = client.actor("maxcopell/free-tripadvisor").call(run_input=run_input)
    return client.dataset(run["defaultDatasetId"]).iterate_items()

def getLatLong(dataframe):
    df = pd.DataFrame(columns=['name','location_final'])
    for i in range(len(dataframe)) :
        new_row={
            'name':dataframe.loc[i, "name"],
            'location_final':(
                float(dataframe.loc[i, "location"].get('lat')),
                float(dataframe.loc[i, "location"].get('lng'))
            )
        }
        df = df.append(new_row,  ignore_index = True)
    return df
def joinLatLong(dataframe):
    df = pd.DataFrame(columns=['name','location_final'])
    for i in range(len(dataframe)) :
        new_row={
            'name':dataframe.loc[i, "name"],
            'location_final':(
                float(dataframe.loc[i, "latitude"]),
                float(dataframe.loc[i, "longitude"])
            )
        }
        df = df.append(new_row,  ignore_index = True)
    return df

In [6]:
BookingScrapperData=BookingScrapper("Paris",5)
BookingScrapperDataDf = pd.DataFrame(BookingScrapperData)

In [54]:
TripadvisorScrapperData=TripadvisorScrapper("Paris",100)
TripadvisorScrapperDataDf = pd.DataFrame(TripadvisorScrapperData)

In [ ]:
BookingScrapperDataDf2 = BookingScrapperDataDf[['type','url','name','stars','reviews','rating','image','location']].copy()
BookingScrapperDataDf3=pd.merge(BookingScrapperDataDf2,getLatLong(BookingScrapperDataDf2),on='name')
BookingScrapperDataDf4= BookingScrapperDataDf3[['type','url','name','stars','reviews','rating','image','location_final']].copy()

In [ ]:
TripadvisorScrapperDataDf2 = TripadvisorScrapperDataDf[['name','webUrl','address','rankingPosition','latitude','longitude']].copy()
TripadvisorScrapperDataDf3=pd.merge(TripadvisorScrapperDataDf2,joinLatLong(TripadvisorScrapperDataDf2),on='name')
TripadvisorScrapperDataDf4= TripadvisorScrapperDataDf3[['address','rankingPosition','location_final']].copy()

In [118]:
MergedFrame=pd.merge(BookingScrapperDataDf4,TripadvisorScrapperDataDf4,on='location_final')
tableFait = MergedFrame[['url','rating','stars','reviews','rankingPosition']].copy()
dimension1=MergedFrame[['url','name','image','type','address','location_final']].copy()

In [119]:
hostname="localhost"
dbname="dw"
uname="root"
pwd="rootroot"

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))
conn = mysql.connector.connect(user='root', password='rootroot', host='localhost', database='dw')
cursor = conn.cursor()

cursor.execute("DROP TABLE fait")
cursor.execute("DROP TABLE  dimension1")

tableFait.to_sql('fait', engine, index=False)
dimension1.to_sql('dimension1', engine, index=False)

conn.close()